# ExCAPEDB v1 fix

ExCAPEDB is described in

> Sun J, Jeliazkova N, Chupakin V, Golib-Dzib J-F, Engkvist O, Carlsson L, Wegner J, Ceulemans H, Georgiev I, Jeliazkov V, Kochev N, Ashby TJ, Chen H: ExCAPE-DB: an integrated large scale dataset facilitating Big Data analysis in chemogenomics. J Cheminform 2017, 9:17.  https://jcheminf.biomedcentral.com/articles/10.1186/s13321-017-0203-5

The original release at Zenodo https://zenodo.org/record/173258#.XECnKVUzZlY contains a number of broken SMILES. This notebook:
- Downloads the file from Zenodo
- Check if RDKit is happy with the SMILES
- If not, retrieves the MOL files from the original source (PubChem or ChEMBL)
- Runs standadization with ambit-cli
- Patches the Zenodo file with the new SMILES

In [2]:
import os,path
import datetime
import urllib.request
import logging
from logging.config import fileConfig
from  rdkit  import  Chem
fileConfig('logging.ini')
%run settings.py
logger= logging.getLogger()
logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

2019-01-18 13:33:12,584  DEBUG    Started at nt 	2019-01-18 13:33:12.584413


In [4]:
import pandas as pd

class ExCAPEDB():
    def __init__(self,path, file_in='pubchem.chembl.dataset4publication_inchi_smiles.tsv.xz',file_patched='pubchem.chembl.dataset4publication_inchi_smiles_v2.tsv'):
        self.zenodo = "https://zenodo.org/record/173258/files/pubchem.chembl.dataset4publication_inchi_smiles.tsv.xz?download=1"
        self.path = path
        self.file = "{}/{}".format(path,file_in)
        self.file_patched = "{}/pubchem.chembl.dataset4publication_inchi_smiles_v2.tsv".format(path,file_patched)
        self.id_tag='Original_Entry_ID'
        self.db_tag='DB'
        self.smi_tag='SMILES'
        self.inchikey_tag='Ambit_InchiKey'
        self.inchi_tag='InChI'
        self.id_index  = None
        self.db_index  = None
        self.smi_index = None
        self.inchikey_index = None
        self.inchi_index=None
        self.smiles_errors = {}
        
        

    def getSmiles(self,line):
        if line is None:
            return None
        return line[self.smi_index]
    
    def getDB(self,line):
        if line is None:
            return None
        db = line[self.db_index]
        if db.startswith("pubchem"):
            db="pubchem"
        return db    
    
    def getIdentifier(self,line):
        if line is None:
            return None
        return line[self.id_index]    
    
    def read(self,process=None,_max_records=100,process_header=None):
        import lzma

        header=[]
        r=0
        with lzma.open(excapedb.file, mode='rt') as file:
            prev_line=None
            for line in file:
                line = line.strip().split("\t")
                if r==0:
                    header=line
                    self.id_index=line.index(self.id_tag)
                    self.db_index=line.index(self.db_tag)
                    self.smi_index=line.index(self.smi_tag)
                    self.inchi_index=line.index(self.inchi_tag)
                    self.inchikey_index=line.index(self.inchikey_tag)
                    if not process_header is None:
                        process_header(header)
                    print(line)
                else:    
                    if process is None:
                        print(line[self.id_index])
                        print(line[self.db_index])
                        print(line[self.smi_index])
                    else:
                        process(r,line,prev_line)
                    prev_line=line    
                    if _max_records>0 and r>_max_records:
                        break
                r=r+1        
                
    def replace(self,line, smiles,inchikey,inchi)    :
        line[self.smi_index]=smiles
        line[self.inchikey_index]=inchikey
        line[self.inchi_index]=inchi
        return line
        
    def check_smiles(self,num,line,prev_line=None):
        is_error=False
        _id = self.getIdentifier(line)
        _db = self.getDB(line)
        #we have checked this one already
        if (_id == self.getIdentifier(prev_line)):
            return
        
        try:
            if _id in self.smiles_errors[_db]:
                return
        except:
            pass

        try:
            smiles = self.getSmiles(line)
            m = Chem.MolFromSmiles(smiles)
            if m == None :
                is_error=True
        except ValueError as e:
            is_error=True
            
        if is_error:    
            if not _db in self.smiles_errors:
                self.smiles_errors[_db] = []    
            
            self.smiles_errors[self.getDB(line)].append(_id)  
            
    def error_file(self,db):
        return "{}/errors_{}.txt".format(self.path,db)
        
    def write_errors(self):
        for db in self.smiles_errors:
            tmp=pd.DataFrame({"id" : self.smiles_errors[db]})
            file = self.error_file(db)
            print(file)
            tmp.to_csv(file,index=False,sep="\t")
            
    def read_errors(self):
        for db in ["pubchem","chembl20"]:
            file = self.error_file(db)
            tmp = pd.read_csv(file,sep="\t")            
            self.smiles_errors[db]=tmp["id"].values
    #print(excapedb.getDB(line))
    #print(excapedb.getIdentifier(line))
    #print(excapedb.getSmiles(line))        
#if os.path.isfile(excapedb[""]) 

excapedb = ExCAPEDB(local_path)
if not os.path.isfile(excapedb.file):
    urllib.request.urlretrieve(excapedb.zenodo, excapedb.file)
#print(excapedb.file)

## Check SMILES


In [ ]:
if not os.path.isfile(excapedb.error_file("pubchem")):
    excapedb.read(excapedb.check_smiles,_max_records=-1)
    #Write identifiers of error SMILES into files (per data source)
    excapedb.write_errors()
else: # already stored , read them back                      
    excapedb.read_errors()

In [ ]:
print(excapedb.smiles_errors)


## Retrieve from data source

In [ ]:
import requests

def retrieve(dbsource,_max_records=-1):
    for db in dbsource:
        try:
            os.mkdir("{}/{}".format(excapedb.path,db))
        except:
            pass
                     
        n=0
        if db == "chembl20":
            #chembl 'SDF' files are neither SDF nor mol ... 
            #they have no $$$$ record tag, but have properties after M END ...
            url="https://www.ebi.ac.uk/chembl/api/data/molecule/{}.sdf"
            file_path="{}/{}/{}.sdf"
            suffix="$$$$"         
        else:
            url="https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/record/sdf"
            file_path="{}/{}/{}.sdf"
            suffix=None
        
        for _id in excapedb.smiles_errors[db]:
            file=file_path.format(excapedb.path,db,_id)
            #logger.debug(file)
            if not os.path.isfile(file):
                #urllib.request.urlretrieve(url.format(_id), file)
                try:
                    r = requests.get(url.format(_id))
                    if r.status_code == 200:
                        with open(file, 'wb') as f:  
                            f.write(r.content)
                            if not suffix is None:
                                f.write(suffix.encode())
                    else:
                        logger.error(url.format(_id))
                except Exception as err:
                    logger.error("%s %s",err, url.format(_id))
                    
            n=n+1
            if _max_records>0 and n>_max_records:
                break

excapedb.read_errors()        
dbsource=excapedb.smiles_errors.keys()
retrieve(["chembl20"])            

These CHEMBL20 structures do not exists anymore in ChEMBL
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL554792.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL561096.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL1628593.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL561095.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL3210799.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL76164.sdf

The mol files are extracted from MySQL ChEMBL20 dump instead of online query

```
 SELECT chembl_id, concat(molfile,"\n> <chembl_id>\n",chembl_id,"\n\n") FROM molecule_dictionary join  compound_structures using(molregno)
 where chembl_id in ("CHEMBL554792","CHEMBL561096","CHEMBL1628593","CHEMBL561095","CHEMBL3210799","CHEMBL76164")
``` 

## Standardization

- Download ambitcli-4.0.0-20181208.jar from  https://sourceforge.net/projects/ambit/files/Ambit2/AMBIT%20applications/ambitcli/ambitcli-4.0.0/
- Run standardization as in  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5340785/

In [ ]:
#Runs ambitcli as external file. Java has to be installed and on the path
import subprocess

tags = {"pubchem" : "-d tag_tokeep=PUBCHEM_CID,PUBCHEM_OPENEYE_ISO_SMILES", "chembl20" : "-d tag_tokeep=chembl_id"}
ambit_bin="{}/ambitcli-4.0.0-20181208.jar".format(excapedb.path)

#for db in ["chembl20","pubchem"]:
for db in ["chembl20"]:
    infile  = "{}/{}/".format(excapedb.path,db);
    outfile_step1 = "{}/{}_step01.txt".format(excapedb.path,db);
    outfile_step2 = "{}/{}_step02.txt".format(excapedb.path,db);
    logfile= "{}/{}.log".format(excapedb.path,db)
    
    if not os.path.isfile(outfile_step1):
        cmp_step1 = "java -jar {} -a standardize -m post -d page=0 -d pagesize=-1 –d tautomers=false {} -d smilescanonical=false –d smiles=true -d inchi=true -i {} -o {}".format(ambit_bin,tags[db],infile,outfile_step1)
        print(cmp_step1)
        subprocess.run(cmp_step1)    

    if not os.path.isfile(outfile_step2):
        cmp_step2 = "java -jar {} -a standardize -m post -d page=0 -d pagesize=-1 –d tag_smiles=AMBIT_SMILES -d tag_inchi=AMBIT_InChI -d tag_inchikey=AMBIT_InChIKey –d tautomers=true -d splitfragments=true -d implicith=true -d smilescanonical=false -d smiles=true -d inchi=true -d neutralise=true -d isotopes=true {} -i {} -o {}".format(ambit_bin,tags[db],outfile_step1,outfile_step2)
        print(cmp_step2)
        subprocess.run(cmp_step2)    

## Patch the distribution

In [5]:
smiles_std={}
ids = {"pubchem" : "PUBCHEM_CID", "chembl20" : "chembl_id"}
for db in ["chembl20","pubchem"]:
    outfile_step2 = "{}/{}_step02.txt".format(excapedb.path,db);
    smiles_std[db] = pd.read_csv(outfile_step2,sep="\t")
    
    display(smiles_std[db].head())

In [6]:
db="chembl20"
_id=ids[db]
print(_id)
_value='CHEMBL104260'
tmp=smiles_std[db]
tmp.loc[tmp[_id]==_value]["SMILES"].values[0]
print(excapedb.file_patched)

chembl_id
G://ChemicalData/EXCAPE/excapedb_fix/pubchem.chembl.dataset4publication_inchi_smiles_v2.tsv


In [7]:
def process_header(line):
    patched_fh.write("\t".join(line))
    patched_fh.write("\tupdated")
    patched_fh.write("\n")
    
def patch_smiles(num,line,prev_line=None):
    _id = excapedb.getIdentifier(line)
    _db = excapedb.getDB(line)
    _smiles = excapedb.getSmiles(line)
    if (_db=="pubchem"):
        _id=int(_id)
    
    updated=""
    if _id in excapedb.smiles_errors[_db]:
        if num % 100000 == 0:
            logger.info(num)
        #print(_smiles)
        if _db in smiles_std:
            try:
                _tag=ids[_db]
                tmp=smiles_std[_db]
                rows = tmp.loc[tmp[_tag]==_id]
                
                new_smiles=rows['SMILES'].values[0]
                new_inchi=rows['InChI'].values[0]
                new_inchikey=rows['InChIKey'].values[0]
                line = excapedb.replace(line, new_smiles,new_inchikey,new_inchi)
                updated="v2"
            except Exception as err:
                logger.error(err)
                updated="(v2)"
                pass
    #write it        
    patched_fh.write("\t".join(line))
    patched_fh.write("\t")
    patched_fh.write(updated)
    patched_fh.write("\n")
    
            
excapedb.read_errors()        
print( excapedb.smiles_errors["pubchem"])
print( excapedb.smiles_errors["chembl20"])

patched_fh = open(excapedb.file_patched,"w") 
excapedb.read(patch_smiles,_max_records=-1,process_header=process_header)
patched_fh.close()

[  664065  1479464 11834578 ... 16031493 57398070 22519912]
['CHEMBL1498877' 'CHEMBL1598851' 'CHEMBL1428547' ... 'CHEMBL1889571'
 'CHEMBL2047019' 'CHEMBL1714078']
['Ambit_InchiKey', 'Original_Entry_ID', 'Entrez_ID', 'Activity_Flag', 'pXC50', 'DB', 'Original_Assay_ID', 'Tax_ID', 'Gene_Symbol', 'Ortholog_Group', 'InChI', 'SMILES']


2019-01-18 13:42:19,787  INFO     8400000
2019-01-18 13:44:07,300  INFO     10300000
2019-01-18 13:50:49,879  INFO     17900000
2019-01-18 13:51:53,992  INFO     19000000
2019-01-18 13:59:30,374  INFO     27800000
2019-01-18 13:59:36,824  INFO     27900000
2019-01-18 14:27:07,472  INFO     60900000
